# STG single neuron

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from itertools import product as itproduct

In [ ]:
from sys import path as sys_path
from os.path import abspath as os_path_abspath
sys_path.append(os_path_abspath('..'))
import addpaths

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plot_utils as pltu

# Model

In [ ]:
import stim_utils

t0, tmax = 0, 3000
Iamp = 3
stim = stim_utils.Istim(Iamp=Iamp, onset=900, offset=tmax*2, name=f'step{Iamp}')
stim.plot(t0=t0, tmax=tmax)

In [ ]:
import stg_model

panel = 'b'
neuron = stg_model.stg_model_1n(g_params=panel, stim=stim)
neuron

# Generator

In [ ]:
from data_generator_STG import data_generator_STG
import math_utils

gen = data_generator_STG(
    y0=neuron.y0, t0=t0, tmax=tmax, t_eval_adaptive=math_utils.t_arange(t0, tmax, 1),
    n_samples=40, n_parallel=20, model=neuron, yidxs=np.arange(13),
    gen_det_sols=True, gen_acc_sols=True, acc_same_ts=True,
    base_folder='_data/subunits'
)
gen.update_subfoldername(stim=stim.name, panel=panel)

## Test

In [ ]:
%%time
solver = gen.get_solver(method='EE', step_param=0.1, pert_method='conrad', adaptive=0)
sol = solver.solve(tmax=tmax, n_samples=20)
sol.plot(y_idxs=[0])

# Data

In [ ]:
pert_method = 'conrad'
adaptive = 0
method = 'EE'
step_params = [0.1, 0.025, 0.01]

In [ ]:
for step_param in step_params:
    gen.gen_and_save_data(
        stim=stim, method=method, adaptive=adaptive, step_param=step_param,
        pert_method=pert_method, overwrite=False, plot=False
    )

# Plot

In [ ]:
yidxs = [0,1,11]

## Load data

In [ ]:
from data_loader import data_loader

df = data_loader(gen).load_data2dataframe(
    [(pert_method, adaptive, [method], step_params)], allowgenerror=False, MAEs=False,
)
df.columns

## Plot example

In [ ]:
example_data_row = df.iloc[0]
example_data_row

In [ ]:
def plot_traces(yidxs, axs):
    """Plot traces, assume axis column."""
    stim_ts = np.sort(np.concatenate([np.linspace(t0, tmax, 1001), [stim.onset-1e-6, stim.onset, stim.onset+1e-6]]))
    stim_Is = np.array([neuron.get_Istim_at_t(t) for t in stim_ts])
    pltu.plot_stim_on_trace_plot(ax=axs[0], ts=stim_ts, stim=stim_Is, stim_y0=60, stim_y1=77)
    axs[0].set_ylim(-85, 55)

    for ax, yidx in zip(axs, yidxs):
        pltu.plot_sample_trace(
            ax=ax, ts=example_data_row.acc_ts, ys=example_data_row.acc_ys[:,yidx], label='ref.', intpol_dt=0.5)
        pltu.plot_mean_and_uncertainty(
            ax=ax, ts=example_data_row.ts, ys=example_data_row.ys[:,:,yidx],
            intpol_dt=0.5, smpidxs=[0]
        )
        ax.set_ylabel(r"$\mathrm{" + f"{neuron.get_y_names()[yidx]}(t)" + "}$")
        if yidx != yidxs[-1]: ax.set_xticklabels([])
    axs[-1].set_xlabel('Time (s)')
    for ax in axs: pltu.scale_ticks(ax, scale=1e-3)

In [ ]:
fig, axs = pltu.subplots(1, len(yidxs), squeeze=False, ysizerow=0.8, xsize='text')
plot_traces(yidxs=yidxs, axs=axs.flat)
plt.tight_layout()

## Runtime vs MEA

In [ ]:
import metric_utils

def plot_runtime_vs_MAE_SR(ax, df, yidxs, lss=['-','--',':'], markers=['o', 'd', 'v'], legend=True):
    """Plot summary. Assumes 2d-axs"""
    
    assert np.unique(df.method).size == 1
    assert np.unique(df.adaptive).size == 1
    assert np.unique(df.pert_method).size == 1
    assert np.unique(df.pert_param).size == 1
    
    df = df.sort_values(by=['step_param'], ascending=False)
    step_params = np.flip(np.unique(df.step_param))
    
    ### Plot data ###
    for ii, yidx in enumerate(yidxs):
        label = r"norm. $\mathrm{" + f"{neuron.get_y_names()[yidx]}(t)" + "}$"

        # Compute normalized data
        MAEs = []
        for _, data_row in df.iterrows():
            samples = data_row['ys'][:,:,yidx]
            target = data_row['acc_ys'][:,yidx]

            targetmin, targetmax = target.min(), target.max()

            samples = (samples - targetmin) / (targetmax - targetmin)
            target = (target - targetmin) / (targetmax - targetmin)

            MAE_SR = metric_utils.compute_sample_target_distances(samples=samples, target=target)
            MAEs.append(MAE_SR)

        # Plot data
        pltu.plot_percentiles(
            ax,
            positions=step_params, data=MAEs,
            marker=markers[ii],
            color=pltu.neuron2color((ii+2)%3),
            mean_kw=dict(ls=lss[ii], label=label),
            line_kw=dict(capsize=4, alpha=0.7, capthick=2),
            connect_alpha=0.7,
            showflier=False,
        )

    ### Decorate ###
    ax.set(xscale='log', yscale='log')
    ax.invert_xaxis()
    ax.grid(True, axis='y', alpha=.3, c='k', lw=plt.rcParams['ytick.major.width'])
    
    if legend: ax.legend(loc='lower left', handlelength=2.3, bbox_to_anchor=(0.01,-0.02))

    pltu.set_labs(ax, xlabs='Step-size (ms)')
    pltu.set_labs(ax, ylabs=r'MAE$_\mathrm{SR}$')

In [ ]:
fig, ax = pltu.subplots(1, 1, ysizerow=2, yoffsize=0.3, squeeze=True)
plot_runtime_vs_MAE_SR(ax, df, yidxs=yidxs)

## Plot both on the same figure

In [ ]:
figshape = (3, 5)

fig = plt.figure(figsize=(pltu.TEXT_WIDTH, 2))

ax1 = plt.subplot2grid(figshape, (0, 0), colspan=3, rowspan=1)
ax2 = plt.subplot2grid(figshape, (1, 0), colspan=3, rowspan=1)
ax3 = plt.subplot2grid(figshape, (2, 0), colspan=3, rowspan=1)

### Plot ###
plot_traces(yidxs=yidxs, axs=[ax1, ax2, ax3])
ax4 = plt.subplot2grid(figshape, (0, 3), colspan=2, rowspan=3)

plot_runtime_vs_MAE_SR(ax=ax4, df=df, yidxs=yidxs)

### Decorate ###
pltu.move_xaxis_outward([ax1, ax2, ax3, ax4], scale=2)
pltu.set_labs([ax1, ax2, ax3, ax4], panel_nums='auto', panel_num_space=10, panel_num_va='top')
for ax in [ax1, ax2]: ax.set_xticklabels([])

sns.despine()
fig.align_labels()
pltu.tight_layout(h_pad=-0.3, w_pad=1)

ax3.legend(loc='upper right', borderpad=0.1, borderaxespad=0.3, frameon=True, bbox_to_anchor=(0,0,1,1.55))

pltu.savefig("STG1_subunits")
plt.show()

pltu.show_saved_figure(fig)